### Importing the Libraries

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

c:\Users\DIVYA GOEL\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\DIVYA GOEL\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\DIVYA GOEL\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Users\DIVYA GOEL\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
tf.__version__ 

'2.9.1'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [6]:
training_set = train_datagen.flow_from_directory('F:/capstone project/Sign-Language-To-Text-Conversion/dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 42000 images belonging to 35 classes.


In [7]:
test_set = test_datagen.flow_from_directory('F:/capstone project/Sign-Language-To-Text-Conversion/dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 42000 images belonging to 35 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [8]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [9]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [10]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [11]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [12]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [13]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=35, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [14]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [15]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [16]:
classifier.fit(training_set,
                  epochs = 10,
                  validation_data = test_set)

Epoch 1/10
4200/4200 [==============================] - 1645s 391ms/step - loss: 0.4979 - accuracy: 0.8442 - val_loss: 2.2415e-04 - val_accuracy: 1.0000
Epoch 2/10
4200/4200 [==============================] - 1683s 401ms/step - loss: 0.1197 - accuracy: 0.9623 - val_loss: 9.1284e-06 - val_accuracy: 1.0000
Epoch 3/10
4200/4200 [==============================] - 2052s 489ms/step - loss: 0.0786 - accuracy: 0.9751 - val_loss: 4.5291e-05 - val_accuracy: 1.0000
Epoch 4/10
4200/4200 [==============================] - 2570s 612ms/step - loss: 0.0639 - accuracy: 0.9803 - val_loss: 2.7521e-07 - val_accuracy: 1.0000
Epoch 5/10
4200/4200 [==============================] - 5208s 1s/step - loss: 0.0529 - accuracy: 0.9843 - val_loss: 4.5826e-07 - val_accuracy: 1.0000
Epoch 6/10
4200/4200 [==============================] - 3658s 871ms/step - loss: 0.0469 - accuracy: 0.9868 - val_loss: 1.9965e-08 - val_accuracy: 1.0000
Epoch 7/10
4200/4200 [==============================] - 3779s 900ms/step - loss: 0.04

#### Saving the Model

In [17]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved
